In [6]:
import pandas as pd
import math

df_data = pd.read_parquet('transaction_fraud_data.parquet')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\alexk\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\alexk\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\alexk\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

AttributeError: _ARRAY_API not found

In [2]:
exchange_rates = pd.read_parquet('historical_currency_exchange.parquet', engine='fastparquet')

In [3]:
print(df_data)  

        transaction_id customer_id       card_number  \
0          TX_a0ad2a2a  CUST_72886  6646734767813109   
1          TX_3599c101  CUST_70474   376800864692727   
2          TX_a9461c6d  CUST_10715  5251909460951913   
3          TX_7be21fc4  CUST_16193   376079286931183   
4          TX_150f490b  CUST_87572  6172948052178810   
...                ...         ...               ...   
7483761    TX_58a762fa  CUST_11344   377055034743699   
7483762    TX_dabcf671  CUST_38961   373625915641065   
7483763    TX_ab567eb4  CUST_75576  6702342842796660   
7483764    TX_5ae61846  CUST_82508   374615717295581   
7483765    TX_54fccca9  CUST_20139   373017396639208   

                         timestamp vendor_category vendor_type  \
0       2024-09-30 00:00:01.034820      Restaurant   fast_food   
1       2024-09-30 00:00:01.764464   Entertainment      gaming   
2       2024-09-30 00:00:02.273762         Grocery    physical   
3       2024-09-30 00:00:02.297466             Gas       major 

In [ ]:
# 1.Расчет доли мошеннических транзакций
fraud_percentage = df_data['is_fraud'].mean() * 100
total_transactions = len(df_data)
fraud_count = df_data['is_fraud'].sum()

print(f"Общее количество транзакций: {total_transactions:,}")
print(f"Мошеннических транзакций: {fraud_count:,} ({fraud_percentage:.2f}%)")

Общее количество транзакций: 7,483,766
Мошеннических транзакций: 1,494,719 (19.97%)


In [20]:
df=df_data.copy()

In [21]:
fraud_count_per_country = df.groupby('country')['is_fraud'].count().reset_index(name='count')
print(fraud_count_per_country)

      country   count
0   Australia  496695
1      Brazil  804800
2      Canada  532632
3      France  541287
4     Germany  524464
5       Japan  527393
6      Mexico  785704
7     Nigeria  849840
8      Russia  793730
9   Singapore  588668
10         UK  538493
11        USA  500060


In [68]:
# 51. В каких топ-5 странах зафиксировано больше всего мошеннических транзакций?
# 2. Расчет доли мошенничества в каждой стране (от общего числа транзакций в стране)
fraud_count_per_country = df.groupby('country').agg(
    total_count=('is_fraud', 'size'),
    fraud_count=('is_fraud', 'sum'),
    fraud_rate=('is_fraud', 'mean')
).reset_index().sort_values(by='fraud_count', ascending=False).head(5)

top_5_countries = ','.join(fraud_count_per_country['country'].astype(str))
print(top_5_countries)

Russia,Mexico,Brazil,Nigeria,Australia


In [31]:
total_hours = (df['timestamp'].max() - df['timestamp'].min()).total_seconds() / 3600

print(f"часов суммарно: {total_hours}")

# Общее количество транзакций
total_transactions = len(df)
print(f"Общее количество транзакций: {total_transactions}")
# Количество уникальных клиентов
unique_clients = df['customer_id'].nunique()

print(f"Количество уникальных клиентов: {unique_clients}")

# Количество уникальных часов в данных (если timestamp покрывает не все 24 часа)
unique_hours = df['timestamp'].dt.hour.nunique()

# Среднее количество транзакций на клиента за час
average = total_transactions / (unique_clients * total_hours)
print(f"Среднее количество транзакций на клиента за час: {average:.2f}")

часов суммарно: 743.9994630736111
Общее количество транзакций: 7483766
Количество уникальных клиентов: 4869
Среднее количество транзакций на клиента за час: 2.07


In [ ]:
#53. Сколько транзакций в среднем совершает один клиент за час?
# 1. Для каждого клиента вычисляем:
#    - количество его транзакций
#    - количество уникальных часов активности (каждый час с хотя бы одной транзакцией = 1 час)
client_stats = df.groupby('customer_id').agg(
    transaction_count=('transaction_id', 'count'),
    active_hours=('timestamp', lambda x: x.dt.floor('H').nunique())
)

# 2. Вычисляем среднее количество транзакций в час для каждого клиента
client_stats['tx_per_hour'] = client_stats['transaction_count'] / client_stats['active_hours']

# 3. Усредняем по всем клиентам и округляем ВНИЗ до 1 знака
average_tx_per_hour = client_stats['tx_per_hour'].mean()
result = math.floor(average_tx_per_hour)

print(f"{result:.1f}")

C:\Users\alexk\AppData\Local\Temp\ipykernel_13384\1023715909.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  active_hours=('timestamp', lambda x: x.dt.floor('H').nunique())
C:\Users\alexk\AppData\Local\Temp\ipykernel_13384\1023715909.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  active_hours=('timestamp', lambda x: x.dt.floor('H').nunique())
C:\Users\alexk\AppData\Local\Temp\ipykernel_13384\1023715909.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  active_hours=('timestamp', lambda x: x.dt.floor('H').nunique())
C:\Users\alexk\AppData\Local\Temp\ipykernel_13384\1023715909.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  active_hours=('timestamp', lambda x: x.dt.floor('H').nunique())
C:\Users\alexk\AppData\Local\Temp\ipykernel_13384\1023715909.py:6: FutureWar

2.4


C:\Users\alexk\AppData\Local\Temp\ipykernel_13384\1023715909.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  active_hours=('timestamp', lambda x: x.dt.floor('H').nunique())
C:\Users\alexk\AppData\Local\Temp\ipykernel_13384\1023715909.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  active_hours=('timestamp', lambda x: x.dt.floor('H').nunique())
C:\Users\alexk\AppData\Local\Temp\ipykernel_13384\1023715909.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  active_hours=('timestamp', lambda x: x.dt.floor('H').nunique())
C:\Users\alexk\AppData\Local\Temp\ipykernel_13384\1023715909.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  active_hours=('timestamp', lambda x: x.dt.floor('H').nunique())
C:\Users\alexk\AppData\Local\Temp\ipykernel_13384\1023715909.py:6: FutureWar

In [ ]:
#52. Какова доля мошенничества среди транзакций у продавцов с высоким риском

# Фильтрация высокорисковых продавцов
high_risk_transactions = df[df['is_high_risk_vendor']]

# Расчет доли мошенничества
fraud_ratio = high_risk_transactions['is_fraud'].mean() * 100

# Округление ВВЕРХ до 1 знака после запятой
rounded_ratio = math.ceil(fraud_ratio * 10) / 10

print(f"Доля мошенничества у высокорисковых продавцов: {rounded_ratio:.1f}%")

Доля мошенничества у высокорисковых продавцов: 20.0%


In [61]:
d1=df[df['is_high_risk_vendor'] == True]
print(d1.shape[0])

print(d1[d1['is_fraud'] == True].shape[0])
374367


print(d1[d1['is_fraud'] == True].shape[0]/d1.shape[0])

1871963
374367
0.19998632451602943


In [60]:
d1=df.copy()
print(d1.shape[0])

print(d1[d1['is_fraud'] == True].shape[0])

print(d1[d1['is_fraud'] == True].shape[0]/d1.shape[0])

7483766
1494719
0.19972818498066347


In [ ]:
#54 В каком городе наибольшая средняя сумма транзакций?

# Группировка по городам и расчет средней суммы транзакций
city_avg = df.groupby('city', observed=True)['amount'].mean()

# Находим город с максимальной средней суммой
top_city = city_avg.idxmax()
avg_amount = city_avg.max()

print(f"Город с наибольшей средней суммой транзакций: {top_city}")
print(f"Средняя сумма: ${avg_amount:,.2f}")

Город с наибольшей средней суммой транзакций: Unknown City
Средняя сумма: $51,315.78


In [73]:
# Удаляем строки с неизвестным городом (фильтруем "Unknown City")
df = df[df['city'] != 'Unknown City']

# Группируем по городам и вычисляем среднюю сумму транзакций
city_avg = df.groupby('city')['amount'].mean()

# Находим город с максимальной средней суммой
top_city = city_avg.idxmax()
top_avg_amount = city_avg.max()

print(f"Город с наибольшей средней суммой транзакций: {top_city}")

Город с наибольшей средней суммой транзакций: New York


In [ ]:
#55 В каком городе выше всего средний чек по операциям, связанным с fast_food?

# Фильтрация: только fast_food и известные города
fast_food_trans = df[
    (df['vendor_type'] == 'fast_food') & 
    (df['city'] != 'Unknown City')
]

# Группировка по городам и расчет среднего чека
city_avg = fast_food_trans.groupby('city')['amount'].mean()

# Город с максимальным средним чеком
top_city = city_avg.idxmax()
print(top_city)

Chicago


In [4]:
df=df_data.copy()

In [8]:
#56.Каково среднее для всех немошеннических операций при пересчете в доллары США?

# Фильтрация немошеннических операций
legit = df[df['is_fraud'] == False].copy()

# Преобразование дат в обеих таблицах к единому формату
legit['date'] = pd.to_datetime(legit['timestamp'].dt.date)
exchange_rates['date'] = pd.to_datetime(exchange_rates['date'])

# Объединение по дате
merged = legit.merge(exchange_rates, on='date', how='left')

# Конвертация в USD
def convert_to_usd(row):
    if row['currency'] == 'USD':
        return row['amount']
    return row['amount'] / row[row['currency']]

merged['amount_usd'] = merged.apply(convert_to_usd, axis=1)

# Расчет среднего с округлением вверх
avg_usd = math.ceil(merged['amount_usd'].mean())

print(f"Средняя сумма немошеннических транзакций: {avg_usd} USD")

Средняя сумма немошеннических транзакций: 460 USD


In [ ]:
print(legit['currency'].unique())

['GBP' 'BRL' 'JPY' 'AUD' 'NGN' 'EUR' 'MXN' 'RUB' 'CAD' 'SGD' 'USD']


In [84]:
print(exchange_rates)

         date       AUD       BRL       CAD       EUR       GBP         JPY  \
0  2024-09-30  1.443654  5.434649  1.351196  0.895591  0.747153  142.573268   
1  2024-10-01  1.442917  5.444170  1.352168  0.897557  0.746956  143.831429   
2  2024-10-02  1.449505  5.425444  1.348063  0.903056  0.752241  143.806861   
3  2024-10-03  1.456279  5.442044  1.351451  0.906018  0.754584  146.916773   
4  2024-10-04  1.460930  5.477788  1.355260  0.906452  0.761891  146.592323   
5  2024-10-05  1.471447  5.457431  1.358726  0.910593  0.761939  148.796290   
6  2024-10-06  1.469948  5.463070  1.357479  0.910655  0.762157  148.725322   
7  2024-10-07  1.469358  5.455679  1.357600  0.911148  0.761611  148.509087   
8  2024-10-08  1.479802  5.495973  1.361858  0.910451  0.763520  147.888136   
9  2024-10-09  1.484282  5.535464  1.365741  0.911611  0.763666  148.274271   
10 2024-10-10  1.487595  5.597276  1.371040  0.913740  0.764894  149.176698   
11 2024-10-11  1.484251  5.583092  1.375260  0.91491

In [17]:
# 60. Определите, сколько клиентов демонстрируют потенциально опасное поведение: 
# для каждого клиента возьмите медианное значение числа уникальных продавцов из показателя активности «за последний час» на момент транзакции и посчитайте, 
# сколько клиентов строго превышают 95-ый квантиль от этого показателя.

# Извлекаем количество уникальных продавцов за последний час для каждой транзакции
client_medians = df.groupby('customer_id')['last_hour_activity.unique_merchants'].median()

# Вычисляем 95-й квантиль по всем клиентам
quantile_95 = client_medians.quantile(0.95)

# Находим клиентов, чья медиана строго превышает 95-й квантиль
risky_clients = client_medians[client_medians > quantile_95]

# Дополнительная информация для анализа
total_clients = len(client_medians)
risky_percentage = len(risky_clients) / total_clients * 100

print(f"Общее количество клиентов: {total_clients}")
print(f"95-й квантиль медианного количества уникальных продавцов: {quantile_95:.1f}")
print(f"Количество клиентов с потенциально опасным поведением: {len(risky_clients)} ({risky_percentage:.1f}% от общего числа)")
print(f"Максимальное медианное значение: {client_medians.max():.1f} уникальных продавцов/час")


Общее количество клиентов: 4869
95-й квантиль медианного количества уникальных продавцов: 100.0
Количество клиентов с потенциально опасным поведением: 229 (4.7% от общего числа)
Максимальное медианное значение: 105.0 уникальных продавцов/час


In [15]:
print(quantile_95)

100.0


In [ ]:
# 57. Каково среднеквадратичное отклонение среди всех немошеннических операций при пересчете в доллары США?

# Фильтрация немошеннических операций
legit_trans = df[df['is_fraud'] == False].copy()

# Приведение дат к одному формату
legit_trans['date'] = pd.to_datetime(legit_trans['timestamp'].dt.date)
exchange_rates['date'] = pd.to_datetime(exchange_rates['date'])

# Объединение с курсами валют
merged = legit_trans.merge(exchange_rates, on='date', how='left')

# Конвертация в USD с проверкой валюты
def convert_to_usd(row):
    if row['currency'] == 'USD':
        return row['amount']
    return row['amount'] / row[row['currency']]

merged['amount_usd'] = merged.apply(convert_to_usd, axis=1)

# Удаление строк с неудачной конвертацией
merged = merged.dropna(subset=['amount_usd'])

# Расчет стандартного отклонения
std_dev = merged['amount_usd'].std()

print(f"Среднеквадратичное отклонение сумм немошеннических транзакций: ${std_dev:,.2f} USD")

Среднеквадратичное отклонение сумм немошеннических транзакций: $417.01 USD


In [ ]:
#58. Каково среднее для всех мошеннических операций при пересчете в доллары США?

# Фильтрация МОШЕННИЧЕСКИХ операций
fraud_trans = df[df['is_fraud'] == True].copy()

# Приведение дат к единому формату
fraud_trans['date'] = pd.to_datetime(fraud_trans['timestamp'].dt.date)
exchange_rates['date'] = pd.to_datetime(exchange_rates['date'])

# Объединение с курсами валют
merged = fraud_trans.merge(exchange_rates, on='date', how='left')

# Конвертация в USD с обработкой ошибок
def convert_to_usd(row):
    if row['currency'] == 'USD':
        return row['amount']
    try:
        return row['amount'] / row[row['currency']]
    except:
        return np.nan

merged['amount_usd'] = merged.apply(convert_to_usd, axis=1)

# Удаление неудачных конвертаций
merged = merged.dropna(subset=['amount_usd'])

# Расчет средней суммы
mean_fraud_usd = math.ceil(merged['amount_usd'].mean())

print(f"Средняя сумма мошеннических операций: ${mean_fraud_usd:,.2f} USD")
print(f"Количество мошеннических транзакций: {len(merged):,}")

Средняя сумма мошеннических операций: $874.61 USD
Количество мошеннических транзакций: 1,494,719


In [22]:
# 59. Каково среднеквадратичное отклонение среди всех немошеннических операций при пересчете в доллары США?

# Фильтрация немошеннических операций
legit_trans = df[df['is_fraud'] == True].copy()

# Приведение дат к одному формату
legit_trans['date'] = pd.to_datetime(legit_trans['timestamp'].dt.date)
exchange_rates['date'] = pd.to_datetime(exchange_rates['date'])

# Объединение с курсами валют
merged = legit_trans.merge(exchange_rates, on='date', how='left')

# Конвертация в USD с проверкой валюты
def convert_to_usd(row):
    if row['currency'] == 'USD':
        return row['amount']
    return row['amount'] / row[row['currency']]

merged['amount_usd'] = merged.apply(convert_to_usd, axis=1)

# Расчет стандартного отклонения
std_dev = math.ceil(merged['amount_usd'].std())

print(f"Среднеквадратичное отклонение сумм мошеннических транзакций: ${std_dev:,.2f} USD")

Среднеквадратичное отклонение сумм мошеннических транзакций: $1,350.00 USD
